In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import dash_html_components as html
import dash_core_components as dcc
%matplotlib inline
mpl.style.use('ggplot')


In [3]:
car=pd.read_csv('Cleaned_Car_data.csv')

In [4]:
car.head()

,Unnamed: 0,name,company,year,Price,kms_driven,fuel_type
0,0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,4,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...,...
811,811,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
812,812,Tata Indica V2,Tata,2009,110000,30000,Diesel
813,813,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
814,814,Tata Zest XM,Tata,2018,260000,27000,Diesel


In [5]:
car=car[car['Price']<6000000]

In [6]:
X=car[['name','company','year','kms_driven','fuel_type']]
y=car['Price']

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)


In [8]:
ohe=OneHotEncoder()
ohe.fit(X[['name','company','fuel_type']])

OneHotEncoder()

In [9]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name','company','fuel_type']),
                                    remainder='passthrough')

In [10]:
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Pe

In [11]:
y_pred=pipe.predict(X_test)
r2_score(y_test,y_pred)

0.6485585785706055

In [12]:
scores=[]
for i in range(1000):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [13]:
scores[np.argmax(scores)]

0.9200894544056878

In [14]:
import pickle

In [15]:
pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))

In [16]:
tp = list(car['fuel_type'].unique())
tp.sort()
fuel_type=[]
for i in tp:
    fuel_type.append({"label":i,"value":i})

In [17]:
name = car['name'].unique()
name = list(name)
name.sort()
model=[]
for i in name:
    model.append({"label":i,"value":i})
# model

In [18]:
company = car['company'].unique()
company = list(company)
company.sort()
brand = []
for i in company:
    brand.append({"label":i,"value":i})

In [43]:
yr = car['year'].unique()
yr = list(yr)
yr.sort()
yr = yr[10:]
year = []
for i in yr:
    year.append({"label":i,"value":i})

In [61]:
import dash
from dash.dependencies import Input,Output
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
        html.H2("Price Prediction"),
        html.Img(src='/assets/data.jpg'),
        html.Hr(),
        html.Div([
            html.Div([
                html.H4('Choose Brand'),
                dcc.Dropdown(
                id = "brand_id",
                    options=brand,
                        value='Audi'
                )],className='dropdown'),
            html.Div([
                html.H4('Choose Model'),
                dcc.Dropdown(
                id = "model_id",
                    options=model,
                        value='Audi A3 Cabriolet'
                )],className='dropdown'),
            html.Div([
                html.H4('Choose Year   '),
                dcc.Dropdown(
                id = "year_id",
                    options=year,
                        value=2009
                ),
            ],className='dropdown'),
            html.Div([
                html.H4('Choose Fuel-type   '),
                dcc.Dropdown(
                id = "ftype_id",
                    options=fuel_type,
                        value='Diesel'
                )],className = 'dropdown'),
            html.Div([
            html.H4('Enter Kms Driven'),
            dcc.Input(
            id = "kms",type='text',placeholder='Enter Kilometer-Driven',value='0')
                ])
        ],className='eight columns'),
    html.Div(
        html.H1(id="output"),style={'color':'white'},className='value'
    )
],className='banner')

@app.callback(
    Output("output","children"),
    Input("model_id","value"),
    Input("brand_id","value"),
    Input("year_id","value"),
    Input("kms","value"),
    Input("ftype_id","value")
)
def update_output(model,brand,year,km,ftype):
    if(brand not in list(model.split())):
        return 'Please choose Correct Brand & Model'
    k=pipe.predict(pd.DataFrame(columns=['name','company','year','kms_driven','fuel_type'],data=np.array([model,brand,year,int(km),ftype]).reshape(1,5)))
    return "%.2f"%k

In [ ]:
if __name__=='__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Envi

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/May/2021 11:53:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2021 11:53:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2021 11:53:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2021 11:53:52] "GET /_favicon.ico?v=1.20.0 HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2021 11:53:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2021 11:54:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2021 11:58:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2021 11:58:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2021 12:00:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2021 12:00:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2021 12:00:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2021 12:01:02] "POST /_dash-update-component H

In [28]:
pipe.predict(pd.DataFrame(columns=['name','company','year','kms_driven','fuel_type'],data=np.array(['Maruti Suzuki Alto','Maruti',2019,100,'Petrol']).reshape(1,5)))

array([316609.61993986])